# This notebook is to collect Data for assessing whether time since TB infection predictor generalizes to Adolescent Acquisition study, or whether current positive predictions are from distribution mismatch between RNA-seq and microarray

# Load Libraries

In [2]:
library(MetaIntegrator)
library(GEOquery)

# Pull the TB datasets together

## Functions to help

In [4]:
gses.array = c(
"GSE19491",


"GSE37250",
"GSE39939",
"GSE39940",

'GSE42825', 'GSE42826' ,'GSE42827', 'GSE42830', 'GSE42831', 'GSE42832'
)


test.gses = gses.array

In [5]:
make.total.dataset = function(data.dir, gses) {
    pheno.list = list()
    exprs.list = list()
    for (GSE in gses) {
        gse.dir = paste(data.dir, GSE, sep="/")
        gene_file = paste(gse.dir, "/", GSE, "_genelevel.csv", sep="")
        pheno_file = paste(gse.dir, "/",  GSE, "_pheno.csv", sep="")
        
        exprs = read.csv(gene_file, row.names=1, header=T)
        pheno = read.csv(pheno_file, row.names=1, header=T)
        
        
        exprs.list[[GSE]] = exprs
        pheno.list[[GSE]] = pheno
    }
    
    results = list()
    results$exprs = exprs.list
    results$pheno.list = pheno.list
    return(results)
    
}

In [6]:
data.dir = "/master/rault/bitbucket/deep_tb/GEO_Warsinske_TB/"

## Read in the GSEs data for test set (validation of recent exposure predictor)

In [7]:
test.data = make.total.dataset(data.dir, test.gses)

In [8]:
str(test.data)

List of 2
 $ exprs     :List of 10
  ..$ GSE19491:'data.frame':	15836 obs. of  498 variables:
  .. ..$ GSM484368: num [1:15836] 14.29 9.51 6.16 7.9 11.51 ...
  .. ..$ GSM484369: num [1:15836] 14.12 9.74 6.1 7.8 11.02 ...
  .. ..$ GSM484370: num [1:15836] 13.81 9.59 6.23 7.85 11.43 ...
  .. ..$ GSM484371: num [1:15836] 13.94 9.68 6.3 7.82 11.59 ...
  .. ..$ GSM484372: num [1:15836] 14.08 9.47 6.02 7.8 11.51 ...
  .. ..$ GSM484373: num [1:15836] 14.24 9.7 6.05 8.11 11.41 ...
  .. ..$ GSM484374: num [1:15836] 14.3 9.49 6.04 7.65 11.71 ...
  .. ..$ GSM484375: num [1:15836] 14.32 9.5 5.95 7.58 11.85 ...
  .. ..$ GSM484376: num [1:15836] 13.96 9.22 5.92 7.81 11.33 ...
  .. ..$ GSM484377: num [1:15836] 14.25 9.25 6.02 7.82 11.8 ...
  .. ..$ GSM484378: num [1:15836] 14.11 9.99 5.78 7.82 11.88 ...
  .. ..$ GSM484379: num [1:15836] 13.96 9.37 6.21 7.66 11.57 ...
  .. ..$ GSM484380: num [1:15836] 13.76 10.16 6.06 7.88 11.32 ...
  .. ..$ GSM484381: num [1:15836] 13.65 9.92 5.93 7.94 11.85 ...
  ..

## Format the test phenotype data

"status" is the TB variable. "hiv.status" is the HIV variable. These are all I care about.

In [9]:
lapply(test.data$pheno.list, function (x) {colnames(x)})

$GSE19491
 [1] "title"                                
 [2] "geo_accession"                        
 [3] "status"                               
 [4] "submission_date"                      
 [5] "last_update_date"                     
 [6] "type"                                 
 [7] "channel_count"                        
 [8] "source_name_ch1"                      
 [9] "organism_ch1"                         
[10] "characteristics_ch1"                  
[11] "characteristics_ch1.1"                
[12] "characteristics_ch1.2"                
[13] "characteristics_ch1.3"                
[14] "characteristics_ch1.4"                
[15] "characteristics_ch1.5"                
[16] "characteristics_ch1.6"                
[17] "characteristics_ch1.7"                
[18] "characteristics_ch1.8"                
[19] "characteristics_ch1.9"                
[20] "characteristics_ch1.10"               
[21] "characteristics_ch1.11"               
[22] "characteristics_ch1.12"               
[23] "characteristics_ch1.13"               
[24] "characteristics_ch1.14"               
[25] "characteristics_ch1.15"               
[26] "characteristics_ch1.16"               
[27] "molecule_ch1"                         
[28] "extract_protocol_ch1"                 
[29] "label_ch1"                            
[30] "label_protocol_ch1"                   
[31] "taxid_ch1"                            
[32] "hyb_protocol"                         
[33] "scan_protocol"                        
[34] "description"                          
[35] "data_processing"                      
[36] "platform_id"                          
[37] "contact_name"                         
[38] "contact_email"                        
[39] "contact_institute"                    
[40] "contact_address"                      
[41] "contact_city"                         
[42] "contact_state"                        
[43] "contact_zip.postal_code"              
[44] "contact_country"                      
[45] "supplementary_file"                   
[46] "data_row_count"                       
[47] "relation"                             
[48] "age.ch1"                              
[49] "bal_culture.ch1"                      
[50] "bal_smear.ch1"                        
[51] "bcg.vaccinated.ch1"                   
[52] "ethnicity.ch1"                        
[53] "exposure_latent.ch1"                  
[54] "gender.ch1"                           
[55] "geographical.region.ch1"              
[56] "healthy.control.ch1"                  
[57] "illness.ch1"                          
[58] "index_case_disease_site.ch1"          
[59] "isolate.sensitivity.ch1"              
[60] "isolate_sensitivity.ch1"              
[61] "modal.x.ray.grade.ch1"                
[62] "region.of.birth.ch1"                  
[63] "smear.of.index.case.ch1"              
[64] "sputum.smear.status.ch1"              
[65] "sputum_culture.ch1"                   
[66] "sputum_smear.ch1"                     
[67] "time.post.initiation.of.treatment.ch1"
[68] "tst.ch1"                              

$GSE37250
 [1] "title"                   "geo_accession"          
 [3] "status"                  "submission_date"        
 [5] "last_update_date"        "type"                   
 [7] "channel_count"           "source_name_ch1"        
 [9] "organism_ch1"            "characteristics_ch1"    
[11] "characteristics_ch1.1"   "characteristics_ch1.2"  
[13] "characteristics_ch1.3"   "molecule_ch1"           
[15] "extract_protocol_ch1"    "label_ch1"              
[17] "label_protocol_ch1"      "taxid_ch1"              
[19] "hyb_protocol"            "scan_protocol"          
[21] "description"             "data_processing"        
[23] "platform_id"             "contact_name"           
[25] "contact_email"           "contact_laboratory"     
[27] "contact_department"      "contact_institute"      
[29] "contact_address"         "contact_city"           
[31] "contact_zip.postal_code" "contact_country"        
[33] "s

### Format GSE19491

- I think some of the TB and control samples may be longitudinal, and in the Duffy et al paper were excluded. Treated TB and anything control shouldn't show recent infection, so for now I will just include them. If needed I can dive deeper into the analysis

In [ ]:
# For GSE19491
# Exclude source_name_ch1 in here
"Whole blood from patient with active TB 12 months after treatment started (i.e. after treatment completed)"
"Whole blood from patient with active TB 2 months after treatment started"

# illness.ch1 is the key phenotype
# anything containing "Control" (i.e. grepl) should be "control". anything containing Latent or LATENT TB should be LTBI
# turn PTB into TB
# keep everything else
#


In [11]:
test.data$pheno.list$GSE19491 = droplevels(test.data$pheno.list$GSE19491[!(test.data$pheno.list$GSE19491$source_name_ch1 %in% 
                                                                c("Whole blood from patient with active TB 12 months after treatment started (i.e. after treatment completed)",
                                                                "Whole blood from patient with active TB 2 months after treatment started")),])
test.data$pheno.list$GSE19491$status = as.character(test.data$pheno.list$GSE19491$illness.ch1)

In [12]:
test.data$pheno.list$GSE19491$status = as.character(ifelse(grepl("Control", test.data$pheno.list$GSE19491$status),
                                              "control", ifelse(grepl("Latent|LATENT", test.data$pheno.list$GSE19491$status),
                                                               "LTBI", ifelse(grepl("PTB", test.data$pheno.list$GSE19491$status),
                                                                             "TB", 
                                                                              ifelse(test.data$pheno.list$GSE19491$status == "ASLE", "SLE",
                                                                                    ifelse(is.na(test.data$pheno.list$GSE19491$status), "control",
                                                                                          test.data$pheno.list$GSE19491$status))))))

In [13]:
test.data$pheno.list$GSE19491$status = ifelse(is.na(test.data$pheno.list$GSE19491$status), "control", test.data$pheno.list$GSE19491$status)
sum(is.na(test.data$pheno.list$GSE19491$status))

[1] 0

In [14]:
test.data$pheno.list$GSE19491$status

[1] "control" "control" "control" "control" "control" "control" "control"
  [8] "control" "control" "control" "control" "control" "TB"      "TB"     
 [15] "TB"      "TB"      "TB"      "TB"      "TB"      "LTBI"    "TB"     
 [22] "control" "TB"      "LTBI"    "control" "control" "LTBI"    "TB"     
 [29] "control" "LTBI"    "TB"      "LTBI"    "LTBI"    "control" "TB"     
 [36] "LTBI"    "control" "TB"      "LTBI"    "TB"      "LTBI"    "TB"     
 [43] "LTBI"    "TB"      "LTBI"    "TB"      "TB"      "LTBI"    "control"
 [50] "TB"      "control" "control" "control" "LTBI"    "control" "LTBI"   
 [57] "TB"      "LTBI"    "control" "LTBI"    "LTBI"    "TB"      "TB"     
 [64] "TB"      "TB"      "LTBI"    "LTBI"    "LTBI"    "LTBI"    "LTBI"   
 [71] "LTBI"    "LTBI"    "LTBI"    "LTBI"    "LTBI"    "LTBI"    "TB"     
 [78] "TB"      "TB"      "TB"      "TB"      "LTBI"    "LTBI"    "LTBI"   
 [85] "LTBI"    "TB"      "TB"      "TB"      "TB"      "LTBI"    "LTBI"   
 [92] "LTBI"    "LTBI"    "LTBI"    "TB"      "TB"      "TB"      "TB"     
 [99] "LTBI"    "LTBI"    "LTBI"    "LTBI"    "LTBI"    "LTBI"    "TB"     
[106] "TB"      "TB"      "LTBI"    "LTBI"    "LTBI"    "LTBI"    "LTBI"   
[113] "TB"      "TB"      "TB"      "TB"      "TB"      "TB"      "TB"     
[120] "TB"      "TB"      "TB"      "TB"      "TB"      "TB"      "TB"     
[127] "TB"      "TB"      "control" "control" "control" "control" "TB"     
[134] "TB"      "TB"      "TB"      "control" "control" "control" "control"
[141] "TB"      "TB"      "TB"      "TB"      "TB"      "TB"      "TB"     
[148] "TB"      "control" "control" "control" "control" "control" "control"
[155] "control" "control" "TB"      "TB"      "TB"      "TB"      "LTBI"   
[162] "LTBI"    "LTBI"    "LTBI"    "control" "control" "TB"      "TB"     
[169] "LTBI"    "LTBI"    "LTBI"    "TB"      "TB"      "control" "control"
[176] "LTBI"    "LTBI"    "LTBI"    "LTBI"    "TB"      "TB"      "TB"     
[183] "TB"      "control" "control" "LTBI"    "LTBI"    "LTBI"    "LTBI"   
[190] "TB"      "TB"      "TB"      "TB"      "control" "control" "control"
[197] "LTBI"    "LTBI"    "LTBI"    "TB"      "TB"      "LTBI"    "TB"     
[204] "TB"      "LTBI"    "control" "TB"      "LTBI"    "control" "control"
[211] "Still"   "Still"   "Still"   "Still"   "Still"   "Still"   "Still"  
[218] "Still"   "Still"   "Still"   "Still"   "Still"   "Still"   "Still"  
[225] "Still"   "Still"   "Still"   "Still"   "Still"   "Still"   "Still"  
[232] "Still"   "Still"   "Still"   "Still"   "Still"   "Still"   "SLE"    
[239] "SLE"     "SLE"     "SLE"     "SLE"     "SLE"     "SLE"     "SLE"    
[246] "SLE"     "SLE"     "SLE"     "SLE"     "SLE"     "SLE"     "SLE"    
[253] "SLE"     "SLE"     "SLE"     "SLE"     "SLE"     "SLE"     "SLE"    
[260] "SLE"     "SLE"     "SLE"     "SLE"     "SLE"     "SLE"     "PSLE"   
[267] "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"   
[274] "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"   
[281] "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"   
[288] "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"   
[295] "PSLE"    "PSLE"    "PSLE"    "PSLE"    "Still"   "Still"   "Staph"  
[302] "Staph"   "Staph"   "Staph"   "Staph"   "Staph"   "Staph"   "Staph"  
[309] "Staph"   "Staph"   "Staph"   "Staph"   "Staph"   "Staph"   "Strep"  
[316] "Strep"   "Strep"   "Strep"   "Strep"   "Strep"   "Strep"   "Strep"  
[323] "Strep"   "Strep"   "Strep"   "Strep"   "Staph"   "Staph"   "Staph"  
[330] "Staph"   "Staph"   "Staph"   "Staph"   "Staph"   "Staph"   "Staph"  
[337] "Staph"   "Staph"   "Staph"   "Staph"   "Staph"   "Staph"   "Staph"  
[344] "Staph"   "Staph"   "Staph"   "Staph"   "Staph"   "Staph"   "Staph"  
[351] "Staph"   "Staph"   "Still"   "Still"   "PSLE"    "PSLE"    "PSLE"   
[358] "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSLE"   
[365] "PSLE"    "PSLE"    "PSLE"    "PSLE"    "PSL

In [16]:
test.data$pheno.list$GSE19491$status == "TB"

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [13]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE FALSE  TRUE FALSE
 [25] FALSE FALSE FALSE  TRUE FALSE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE
 [37] FALSE  TRUE FALSE  TRUE FALSE  TRUE FALSE  TRUE FALSE  TRUE  TRUE FALSE
 [49] FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE
 [61] FALSE  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [73] FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE
 [85] FALSE  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE
 [97]  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE FALSE
[109] FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
[121]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE
[133]  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE
[145]  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[157]  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE
[169] FALSE FALSE FALSE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE
[181]  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE  TRUE
[193]  TRUE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE FALSE  TRUE  TRUE
[205] FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[217] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[229] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[241] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[253] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[265] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[277] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[289] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[301] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[313] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[325] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[337] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[349] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[361] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[373] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[385] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[397] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[409] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[421] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[433] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[445] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[457] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[469] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[481] FALSE FALSE FALSE FALSE

In [17]:
table(test.data$pheno.list$GSE19491$status)


control    LTBI    PSLE     SLE   Staph   Still   Strep      TB 
    133      69      82      28      40      31      12      89 

In [19]:
test.data$pheno.list$GSE19491$hiv.status = "negative"

In [20]:
summary(test.data$pheno.list$GSE19491)

                          title       geo_accession    status         
 CON_LON_test035             :  1   GSM484368:  1   Length:484        
 CON_LON_test035_long_0_022  :  1   GSM484369:  1   Class :character  
 CON_LON_test035_sep_CD4_034 :  1   GSM484370:  1   Mode  :character  
 CON_LON_test035_sep_CD8_035 :  1   GSM484371:  1                     
 CON_LON_test035_sep_Mono_033:  1   GSM484372:  1                     
 CON_LON_test035_sep_Neut_032:  1   GSM484373:  1                     
 (Other)                     :478   (Other)  :478                     
    submission_date    last_update_date  type     channel_count
 Dec 11 2009: 61    Aug 11 2010:387     RNA:484   Min.   :1    
 Dec 12 2009:149    Jan 04 2018: 97               1st Qu.:1    
 Jun 02 2010:193                                  Median :1    
 Jun 03 2010: 81                                  Mean   :1    
                                                  3rd Qu.:1    
                                                

In [24]:
table(test.data$pheno.list$GSE19491$illness.ch1, test.data$pheno.list$GSE19491$source_name_ch1)

                
                 CD4+ cells from healthy control
  ASLE                                         0
  Control                                      0
  Control (BCG-)                               0
  Control (BCG+)                               4
  Latent                                       0
  LATENT TB                                    0
  PSLE                                         0
  PTB                                          0
  Staph                                        0
  Still                                        0
  Strep                                        0
                
                 CD4+ cells from patient with Active TB
  ASLE                                                0
  Control                                             0
  Control (BCG-)                                      0
  Control (BCG+)                                      0
  Latent                                              0
  LATENT TB                               

In [17]:
table(test.data$pheno.list$GSE19491$illness.ch1, test.data$pheno.list$GSE19491$source_name_ch1)

                
                 CD4+ cells from healthy control
  ASLE                                         0
  Control                                      0
  Control (BCG-)                               0
  Control (BCG+)                               4
  Latent                                       0
  LATENT TB                                    0
  PSLE                                         0
  PTB                                          0
  Staph                                        0
  Still                                        0
  Strep                                        0
                
                 CD4+ cells from patient with Active TB
  ASLE                                                0
  Control                                             0
  Control (BCG-)                                      0
  Control (BCG+)                                      0
  Latent                                              0
  LATENT TB                               

### Format GSE37250

- Other disease is a general indication and is not specified, so I will keep it as "other_disease".
- Some individuals are HIV positive in this cohort. I will keep this with TB and examine HIV status later.

In [21]:
test.data$pheno.list$GSE37250$status = as.character(test.data$pheno.list$GSE37250$disease.state.ch1)
test.data$pheno.list$GSE37250$status = ifelse(test.data$pheno.list$GSE37250$status == "active tuberculosis", "TB",
                                              ifelse(test.data$pheno.list$GSE37250$status == "latent TB infection", "LTBI", "other_disease"))

In [22]:
table(test.data$pheno.list$GSE37250$status)


         LTBI other_disease            TB 
          167           175           195 

In [26]:
test.data$pheno.list$GSE37250$hiv.status = as.character(test.data$pheno.list$GSE37250$hiv.status.ch1)
test.data$pheno.list$GSE37250$hiv.status = ifelse(test.data$pheno.list$GSE37250$hiv.status == "HIV negative", "negative", "positive")
table(test.data$pheno.list$GSE37250$hiv.status)


negative positive 
     263      274 

In [27]:
summary(test.data$pheno.list$GSE37250)

                                                  title       geo_accession
 active tuberculosis HIV negative Malawi WB_1_M_39619:  1   GSM914353:  1  
 active tuberculosis HIV negative Malawi WB_1_M_39668:  1   GSM914354:  1  
 active tuberculosis HIV negative Malawi WB_1_M_39827:  1   GSM914355:  1  
 active tuberculosis HIV negative Malawi WB_1_M_39896:  1   GSM914356:  1  
 active tuberculosis HIV negative Malawi WB_1_M_39956:  1   GSM914357:  1  
 active tuberculosis HIV negative Malawi WB_1_M_39985:  1   GSM914358:  1  
 (Other)                                             :531   (Other)  :531  
    status             submission_date    last_update_date  type    
 Length:537         Apr 13 2012:537    Dec 17 2018:537     RNA:537  
 Class :character                                                   
 Mode  :character                                                   
                                                                    
                                               

In [28]:
table(test.data$pheno.list$GSE37250$title)


       active tuberculosis HIV negative Malawi WB_1_M_39619 
                                                          1 
       active tuberculosis HIV negative Malawi WB_1_M_39668 
                                                          1 
       active tuberculosis HIV negative Malawi WB_1_M_39827 
                                                          1 
       active tuberculosis HIV negative Malawi WB_1_M_39896 
                                                          1 
       active tuberculosis HIV negative Malawi WB_1_M_39956 
                                                          1 
       active tuberculosis HIV negative Malawi WB_1_M_39985 
                                                          1 
       active tuberculosis HIV negative Malawi WB_1_M_40029 
                                                          1 
       active tuberculosis HIV negative Malawi WB_1_M_40176 
                                                          1 
       active tuberculo

### Remainder except GSE42831 ('GSE42825', 'GSE42826', 'GSE42827', 'GSE42830', 'GSE42832')

In [29]:
names(test.data$pheno.list)

[1] "GSE19491" "GSE37250" "GSE39939" "GSE39940" "GSE42825" "GSE42826"
 [7] "GSE42827" "GSE42830" "GSE42831" "GSE42832"

In [31]:
to.analyze = c('GSE42825', 'GSE42826', 'GSE42827', 'GSE42830', 'GSE42832')

In [36]:
for (gse in to.analyze) {
    pheno = test.data$pheno.list[[gse]]
    pheno$status = as.character(pheno$disease.state.ch1)
    pheno$status = ifelse(grepl("arcoid", pheno$status), "sarcoidosis",
                         ifelse(grepl("Control", pheno$status), "control",
                               ifelse(grepl("lung cancer", pheno$status), "lung_cancer",
                                     ifelse(grepl("Pneumonia|Baseline", pheno$status), "pneumonia", pheno$status))))
    pheno$hiv.status = as.character("negative")
    test.data$pheno.list[[gse]] = pheno
}

In [37]:
lapply(to.analyze, function(x) {table(test.data$pheno.list[[x]][,"status"])})

[[1]]

    control sarcoidosis          TB 
         23          11           8 

[[2]]

    control lung_cancer   pneumonia sarcoidosis          TB 
         52           8           6          25          11 

[[3]]

pneumonia 
       10 

[[4]]

    control lung_cancer   pneumonia sarcoidosis          TB 
         38           8           8          25          16 

[[5]]

    control sarcoidosis          TB 
         30          30          30 


In [38]:
lapply(to.analyze, function(x) {table(test.data$pheno.list[[x]][,"disease.state.ch1"])})

[[1]]

    Active sarcoidosis                Control Non-active sarcoidosis 
                     6                     23                      5 
                    TB 
                     8 

[[2]]

        Active Sarcoid                Control            lung cancer 
                    16                     52                      8 
Non-active sarcoidosis              Pneumonia                     TB 
                     9                      6                     11 

[[3]]

pneumonia 
       10 

[[4]]

        Active Sarcoid               Baseline                Control 
                    17                      8                     38 
           lung cancer Non-active sarcoidosis                     TB 
                     8                      8                     16 

[[5]]

Control Sarcoid      TB 
     30      30      30 


In [39]:
summary(droplevels(test.data$pheno.list$GSE42830[test.data$pheno.list$GSE42830$disease.state.ch1 == "Baseline",]))

                              title      geo_accession    status         
 Pneumonia Sample 18 training set:1   GSM1050945:1     Length:8          
 Pneumonia Sample 33 training set:1   GSM1050960:1     Class :character  
 Pneumonia Sample 36 training set:1   GSM1050963:1     Mode  :character  
 Pneumonia Sample 38 training set:1   GSM1050965:1                       
 Pneumonia Sample 47 training set:1   GSM1050974:1                       
 Pneumonia Sample 63 training set:1   GSM1050990:1                       
 (Other)                         :2   (Other)   :2                       
    submission_date    last_update_date  type   channel_count
 Dec 10 2012:8      Oct 31 2013:8       RNA:8   Min.   :1    
                                                1st Qu.:1    
                                                Median :1    
                                                Mean   :1    
                                                3rd Qu.:1    
                                    

In [40]:
summary(test.data$pheno.list$GSE42825)

                              title       geo_accession    status         
 Control Sample 10 validation set: 1   GSM1050732: 1    Length:42         
 Control Sample 11 validation set: 1   GSM1050733: 1    Class :character  
 Control Sample 12 validation set: 1   GSM1050734: 1    Mode  :character  
 Control Sample 13 validation set: 1   GSM1050735: 1                      
 Control Sample 14 validation set: 1   GSM1050736: 1                      
 Control Sample 15 validation set: 1   GSM1050737: 1                      
 (Other)                         :36   (Other)   :36                      
    submission_date    last_update_date  type    channel_count
 Dec 10 2012:42     Oct 31 2013:42      RNA:42   Min.   :1    
                                                 1st Qu.:1    
                                                 Median :1    
                                                 Mean   :1    
                                                 3rd Qu.:1    
                      

### Format GSE42831

- All samples are sarcoidosis samples. Just take sarcoidosis pre-treatment

In [44]:
test.data$pheno.list$GSE42831$status = as.character("sarcoidosis")
test.data$pheno.list$GSE42831$hiv.status = as.character("negative")
test.data$pheno.list$GSE42831 = droplevels(test.data$pheno.list$GSE42831[test.data$pheno.list$GSE42831$treament.ch1 == "Pre-treatment",])

In [45]:
summary(test.data$pheno.list$GSE42831)

                                             title      geo_accession
 Sarcoid Sample 10 6011 Pre-treatment time pt. 1:1   GSM1051029:1    
 Sarcoid Sample 11 2010 Pre-treatment time pt. 1:1   GSM1051030:1    
 Sarcoid Sample 14 2140 Pre-treatment time pt. 1:1   GSM1051031:1    
 Sarcoid Sample 15 6009 Pre-treatment time pt. 1:1   GSM1051032:1    
 Sarcoid Sample 7 2099 Pre-treatment time pt. 1 :1   GSM1051033:1    
 Sarcoid Sample 8 2018 Pre-treatment time pt. 1 :1   GSM1051036:1    
 Sarcoid Sample 9 2100 Pre-treatment time pt. 1 :1   GSM1051037:1    
    status             submission_date    last_update_date  type  
 Length:7           Dec 10 2012:7      Oct 31 2013:7       RNA:7  
 Class :character                                                 
 Mode  :character                                                 
                                                                  
                                                                  
                                      

In [46]:
table(test.data$pheno.list$GSE42831$title)


Sarcoid Sample 10 6011 Pre-treatment time pt. 1 
                                              1 
Sarcoid Sample 11 2010 Pre-treatment time pt. 1 
                                              1 
Sarcoid Sample 14 2140 Pre-treatment time pt. 1 
                                              1 
Sarcoid Sample 15 6009 Pre-treatment time pt. 1 
                                              1 
 Sarcoid Sample 7 2099 Pre-treatment time pt. 1 
                                              1 
 Sarcoid Sample 8 2018 Pre-treatment time pt. 1 
                                              1 
 Sarcoid Sample 9 2100 Pre-treatment time pt. 1 
                                              1 

### Format GSE39939

In [47]:
summary(test.data$pheno.list$GSE39939)

              title        geo_accession                   status   
 WB_20_KIL_100091:  1   GSM1249177:  1   Public on May 01 2014:157  
 WB_20_KIL_100360:  1   GSM1249178:  1                              
 WB_20_KIL_100398:  1   GSM1249179:  1                              
 WB_20_KIL_100406:  1   GSM1249180:  1                              
 WB_20_KIL_500086:  1   GSM1249181:  1                              
 WB_20_KIL_500119:  1   GSM1249182:  1                              
 (Other)         :151   (Other)   :151                              
    submission_date    last_update_date  type     channel_count
 Oct 23 2013:157    May 01 2014:157     RNA:157   Min.   :1    
                                                  1st Qu.:1    
                                                  Median :1    
                                                  Mean   :1    
                                                  3rd Qu.:1    
                                                  Max.   :1    


In [49]:
test.data$pheno.list$GSE39939$status = as.character(test.data$pheno.list$GSE39939$illness.ch1)
test.data$pheno.list$GSE39939$status = ifelse(grepl("active tuberculosis", test.data$pheno.list$GSE39939$status), "TB",
                                             ifelse(grepl("other disease", test.data$pheno.list$GSE39939$status), "other_disease",
                                                    "LTBI"))
test.data$pheno.list$GSE39939$hiv.status = as.character(test.data$pheno.list$GSE39939$hiv.status.ch1)
test.data$pheno.list$GSE39939$hiv.status = ifelse(test.data$pheno.list$GSE39939$hiv.status == "HIV negative", "negative", "positive")

In [50]:
test.data$pheno.list$GSE39939$hiv.status

[1] "positive" "positive" "positive" "positive" "positive" "positive"
  [7] "positive" "positive" "positive" "positive" "negative" "negative"
 [13] "negative" "negative" "negative" "negative" "negative" "negative"
 [19] "negative" "negative" "negative" "negative" "negative" "negative"
 [25] "negative" "negative" "negative" "negative" "negative" "negative"
 [31] "negative" "negative" "negative" "negative" "negative" "negative"
 [37] "negative" "negative" "negative" "negative" "negative" "negative"
 [43] "negative" "negative" "negative" "negative" "negative" "negative"
 [49] "negative" "negative" "negative" "negative" "negative" "negative"
 [55] "negative" "negative" "negative" "negative" "negative" "negative"
 [61] "negative" "negative" "negative" "negative" "positive" "positive"
 [67] "positive" "positive" "positive" "positive" "positive" "positive"
 [73] "positive" "positive" "positive" "positive" "positive" "positive"
 [79] "positive" "positive" "positive" "positive" "positive" "positive"
 [85] "positive" "positive" "positive" "positive" "positive" "positive"
 [91] "negative" "negative" "negative" "negative" "negative" "negative"
 [97] "negative" "negative" "negative" "negative" "negative" "negative"
[103] "negative" "negative" "positive" "positive" "negative" "negative"
[109] "positive" "negative" "positive" "positive" "negative" "negative"
[115] "positive" "negative" "negative" "negative" "negative" "negative"
[121] "negative" "positive" "positive" "positive" "positive" "positive"
[127] "negative" "negative" "positive" "positive" "negative" "positive"
[133] "negative" "negative" "positive" "negative" "negative" "negative"
[139] "negative" "positive" "negative" "negative" "negative" "negative"
[145] "negative" "negative" "negative" "positive" "negative" "negative"
[151] "negative" "negative" "negative" "negative" "negative" "positive"
[157] "negative"

In [51]:
table(test.data$pheno.list$GSE39939$status)


         LTBI other_disease            TB 
           14            64            79 

### Format GSE39940

In [52]:
summary(test.data$pheno.list$GSE39940)

           title       geo_accession                   status   
 WB_20_M_40012:  1   GSM981627:  1   Public on May 01 2014:334  
 WB_20_M_40182:  1   GSM981628:  1                              
 WB_20_M_40263:  1   GSM981629:  1                              
 WB_20_M_40376:  1   GSM981630:  1                              
 WB_20_M_40384:  1   GSM981631:  1                              
 WB_20_M_40385:  1   GSM981632:  1                              
 (Other)      :328   (Other)  :328                              
    submission_date    last_update_date  type     channel_count
 Aug 07 2012:334    May 01 2014:334     RNA:334   Min.   :1    
                                                  1st Qu.:1    
                                                  Median :1    
                                                  Mean   :1    
                                                  3rd Qu.:1    
                                                  Max.   :1    
                                

In [54]:
test.data$pheno.list$GSE39940$status = as.character(test.data$pheno.list$GSE39940$disease.status.ch1)
test.data$pheno.list$GSE39940$status = ifelse(test.data$pheno.list$GSE39940$status == "active tuberculosis", "TB",
                                             ifelse(test.data$pheno.list$GSE39940$status == "other disease", "other_disease", "LTBI"))

test.data$pheno.list$GSE39940$hiv.status = as.character(test.data$pheno.list$GSE39940$hiv.status.ch1)
test.data$pheno.list$GSE39940$hiv.status = ifelse(test.data$pheno.list$GSE39940$hiv.status == "HIV negative", "negative", "positive")

In [89]:
table(test.data$pheno.list$GSE39940$status)


         LTBI other_disease            TB 
           54           169           111 

In [55]:
table(test.data$pheno.list$GSE39940$status, test.data$pheno.list$GSE39940$hiv.status)

               
                negative positive
  LTBI                54        0
  other_disease      103       66
  TB                  70       41

## Review that the phenotype formatting is correct and save to csv files (status and GSM identifier)

### Review phenotype data

In [56]:
lapply(test.data$pheno.list, function(x) {table(x$status)})

$GSE19491

control    LTBI    PSLE     SLE   Staph   Still   Strep      TB 
    133      69      82      28      40      31      12      89 

$GSE37250

         LTBI other_disease            TB 
          167           175           195 

$GSE39939

         LTBI other_disease            TB 
           14            64            79 

$GSE39940

         LTBI other_disease            TB 
           54           169           111 

$GSE42825

    control sarcoidosis          TB 
         23          11           8 

$GSE42826

    control lung_cancer   pneumonia sarcoidosis          TB 
         52           8           6          25          11 

$GSE42827

pneumonia 
       10 

$GSE42830

    control lung_cancer   pneumonia sarcoidosis          TB 
         38           8           8          25          16 

$GSE42831

sarcoidosis 
          7 

$GSE42832

    control sarcoidosis          TB 
         30          30          30 


In [57]:
Reduce("+", lapply(test.data$pheno.list, function(x) {sum(x$status == "TB")}))

[1] 539

In [58]:
lapply(test.data$pheno.list, function(x) {sum(x$status == "TB")})

$GSE19491
[1] 89

$GSE37250
[1] 195

$GSE39939
[1] 79

$GSE39940
[1] 111

$GSE42825
[1] 8

$GSE42826
[1] 11

$GSE42827
[1] 0

$GSE42830
[1] 16

$GSE42831
[1] 0

$GSE42832
[1] 30

In [74]:
lapply(test.data$pheno.list, function(x) {table(x$status)})

$GSE19491

control    LTBI    PSLE     SLE   Staph   Still   Strep      TB 
    133      69      82      28      40      31      12      89 

$GSE37250

         LTBI other_disease            TB 
          167           175           195 

$GSE39939

         LTBI other_disease            TB 
           14            64            79 

$GSE39940

         LTBI other_disease            TB 
           54           169           111 

$GSE42826

    control lung_cancer   pneumonia sarcoidosis          TB 
         52           8           6          25          11 

$GSE42827

pneumonia 
       10 

$GSE42830

    control lung_cancer   pneumonia sarcoidosis          TB 
         38           8           8          25          16 

$GSE42831

sarcoidosis 
          7 


### Combine phenotype data and GSM's across all tables, excluding GSE42825 and GSE42832 based on fewer number of genes

In [73]:
test.data$pheno.list$GSE42825 = NULL
test.data$pheno.list$GSE42832 = NULL

In [75]:
test.pheno = data.frame()
for (GSE in test.data$pheno.list) {
    test.pheno = rbind(test.pheno, GSE[,c("status", "hiv.status"), drop=F])
}

In [76]:
dim(test.pheno)
table(test.pheno)

[1] 1726    2

               hiv.status
status          negative positive
  control            223        0
  LTBI               220       84
  lung_cancer         16        0
  other_disease      223      185
  pneumonia           24        0
  PSLE                82        0
  sarcoidosis         57        0
  SLE                 28        0
  Staph               40        0
  Still               31        0
  Strep               12        0
  TB                 335      166

In [77]:
write.csv(test.pheno, "data/Illumina_microarray_GSE_validation/Illumina_microarray_validation_pheno.csv")

## Analyze the number of genes in each microarray and determine how to do overlap

In [63]:
lapply(test.data$exprs, function(x) {dim(x)[1]})

$GSE19491
[1] 15836

$GSE37250
[1] 17879

$GSE39939
[1] 17879

$GSE39940
[1] 17879

$GSE42825
[1] 11147

$GSE42826
[1] 17879

$GSE42827
[1] 17879

$GSE42830
[1] 17879

$GSE42831
[1] 17879

$GSE42832
[1] 12296

In [71]:
lapply(test.data$exprs, function(x) {dim(x)[1]})

$GSE19491
[1] 15836

$GSE37250
[1] 17879

$GSE39939
[1] 17879

$GSE39940
[1] 17879

$GSE42826
[1] 17879

$GSE42827
[1] 17879

$GSE42830
[1] 17879

$GSE42831
[1] 17879

### Exclude GSE42832 and GSE42825 based on smaller number of genes. Keep 19491 as common denominator since these were in my RNA-seq validation

In [65]:
test.data$exprs$GSE42825 = NULL
test.data$exprs$GSE42832 = NULL

## Merge the expression datasets

In [83]:
genes.to.keep = intersect(rownames(test.data$exprs$GSE19491), rownames(test.data$exprs$GSE42830))
length(genes.to.keep)
head(genes.to.keep)

[1] 15821

[1] "EEF1A1"  "GAPDH"   "SLC35E2" "DUSP22"  "RPS28"   "IPO13"

In [80]:
merge.exprs = function(datasets, seed.data) {
    begin = datasets[[seed.data]]
    remaining = setdiff(names(datasets), seed.data)
    print(remaining)
    for (GSE in remaining) {
        begin = merge(begin, datasets[[GSE]], by="row.names", all.x=T)#, suffixes = c("",""))
        rownames(begin) = begin$Row.names
        begin$Row.names = NULL
        
    }
    return(begin)
}

In [84]:
test.data$exprs.fil = lapply(test.data$exprs, function(x) {x[rownames(x) %in% genes.to.keep, ]})

In [85]:
lapply(test.data$exprs.fil, function(x) {dim(x)[1]})

$GSE19491
[1] 15821

$GSE37250
[1] 15821

$GSE39939
[1] 15821

$GSE39940
[1] 15821

$GSE42826
[1] 15821

$GSE42827
[1] 15821

$GSE42830
[1] 15821

$GSE42831
[1] 15821

In [86]:
test.merge = merge.exprs(test.data$exprs.fil, "GSE19491")

[1] "GSE37250" "GSE39939" "GSE39940" "GSE42826" "GSE42827" "GSE42830" "GSE42831"


In [88]:
length(intersect(rownames(test.pheno), colnames(test.merge)))

[1] 1726

In [89]:
test.merge = test.merge[, colnames(test.merge) %in% rownames(test.pheno)]

In [91]:
identical(rownames(test.pheno), colnames(test.merge))

[1] TRUE

In [90]:
dim(test.pheno)
dim(test.merge)

[1] 1726    2

[1] 15821  1726

In [92]:
write.csv(test.merge, "data/Illumina_microarray_GSE_validation/Illumina_microarray_validation_exprs.csv")